In [36]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
import warnings
warnings.filterwarnings('ignore')

In [37]:
folder = ''
temp1 = pd.read_csv(folder+'Task3.csv')

participant ID, 
treatment, part (this will always be 1 within the file), 
rounds (number of rounds available), 
prior (prior probability of red being guilty), 
true_guilty_suspect, 
accused_suspect, 
9 columns for the investigations, 
9 columns for the outcome of the investigation (evidence/no-evidence), 
1 column with at which round the first evidence was observed (0 if no evidence is observed)

In [38]:
folder = ''
temp1 = pd.read_csv(folder+'Task3.csv')
drop_columns = ['num_evidences_total', 'final_evidence', 'bonus_earned', 
        'trial_chosen_for_bonus', 'total_bonus_paid_cents', 'timing_valid', 'timing_error']
num_rounds = 60
temp1 = temp1.drop(columns = drop_columns)
temp1 = temp1.replace('red', 0)
temp1 = temp1.replace('blue', 1)
temp1['true_guilty_suspect'] = 0
temp1['evidence_round'] = 0

for i in range(num_rounds,0,-1):
    temp1.insert(68, 'outcome_'+str(i), np.nan)
temp1.correct_suspect_accused = temp1.correct_suspect_accused.astype(int)

temp = ['investigate_red','investigate_blue','accuse_red','accuse_blue','advance_to_next_trial']

for i in temp:
    temp1 = temp1.replace(i, temp.index(i))
temp1.astype({'suspect_accused': 'int'}).dtypes
temp1.astype({'correct_suspect_accused': 'int'}).dtypes

participant_ID                   int64
trial_no                         int64
setup_cost                     float64
setup_cost_investigate_red     float64
setup_cost_investigate_blue    float64
                                ...   
correct_suspect_accused          int64
part                             int64
treatment                        int64
true_guilty_suspect              int64
evidence_round                   int64
Length: 257, dtype: object

In [39]:
p_count = 0
for key,value in temp1.iterrows():
    temp1['true_guilty_suspect'][key] = int(temp1['suspect_accused'][key]) ^ 1 if temp1['correct_suspect_accused'][key] == 0 else temp1['suspect_accused'][key]
    temp = []
    counter_r = 0
    counter_b = 0
    for i in range(num_rounds):
        if value[i+6] == 0:
            temp.append(1) if value[190+counter_r] == 1 else temp.append(0)
            counter_r += 1
        elif value[i+6] == 1:
            temp.append(1) if value[220+counter_b] == 1 else temp.append(0)
            counter_b += 1
    for i in range(1,len(temp)+1):
        temp1['outcome_'+str(i)][key] = temp[i-1]
    
    temp = temp1.iloc[key, 68:130].to_list()
    temp1['evidence_round'][key] = temp.index(1.0)+1 if 1 in temp else 0
    
    
    temp = temp1.iloc[key, 6:66].to_list()
    temp = [x if x == 1 or x == 0 else np.nan for x in temp]
    temp1.iloc[key, 6:66] = temp

temp1.head()

,participant_ID,trial_no,setup_cost,setup_cost_investigate_red,setup_cost_investigate_blue,red_prior_prob,guilty_suspect_chosen,choice_1,choice_2,choice_3,...,evidence_blue_27,evidence_blue_28,evidence_blue_29,evidence_blue_30,suspect_accused,correct_suspect_accused,part,treatment,true_guilty_suspect,evidence_round
0,1,1,NaN,4.0,6.0,0.7,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,0,1,3,0,0,0
1,1,2,NaN,16.0,24.0,0.1,1.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1,1,3,0,1,0
2,1,3,NaN,6.0,4.0,0.5,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,1,1,3,0,1,0
3,1,4,NaN,16.0,24.0,0.3,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0,1,3,0,0,0
4,1,5,NaN,4.0,6.0,0.3,1.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,1,1,3,0,1,0


In [40]:
print(value[:10])

participant_ID                 16.0
trial_no                       50.0
setup_cost                      NaN
setup_cost_investigate_red     96.0
setup_cost_investigate_blue    64.0
red_prior_prob                  0.7
guilty_suspect_chosen           0.0
choice_1                        0.0
choice_2                        0.0
choice_3                        2.0
Name: 799, dtype: float64


In [41]:
d1 = ['evidence_red_'+str(i) for i in range(1,31)]
d2 = ['evidence_blue_'+str(i) for i in range(1,31)]
d3 = ['choice_'+str(i) for i in range(1,61)]
d4 = ['outcome_'+str(i) for i in range(1,61)]
d5 = ['timing_choice_'+str(i) for i in range(1,63)]
d_columns = d1+d2
temp1 = temp1.drop(columns = d_columns)
temp1 = temp1.rename(columns={"setup_cost_investigate_red": "setup_cost_red",
                             "setup_cost_investigate_blue": "setup_cost_blue"})# 'guilty_suspect_chosen': 'true_guilty_suspect'})
cols = ['participant_ID', 'treatment', 'part', 'trial_no', 
         'setup_cost_red', 'setup_cost_blue', 'red_prior_prob',
        'true_guilty_suspect', 'suspect_accused']+d3+d4+d5+['correct_suspect_accused', 'evidence_round']
temp1 = temp1[cols]
temp1.to_csv('Task3_step2.csv', index=False)

In [42]:
print(temp1.columns)

Index(['participant_ID', 'treatment', 'part', 'trial_no', 'setup_cost_red',
       'setup_cost_blue', 'red_prior_prob', 'true_guilty_suspect',
       'suspect_accused', 'choice_1',
       ...
       'timing_choice_55', 'timing_choice_56', 'timing_choice_57',
       'timing_choice_58', 'timing_choice_59', 'timing_choice_60',
       'timing_choice_61', 'timing_choice_62', 'correct_suspect_accused',
       'evidence_round'],
      dtype='object', length=193)
